In [1]:
%run helper/setup_notebook.ipynb import display_table

Successfully connected to leetcode50 database.


In [2]:
display_table('Visits')

+----------+-------------+
| visit_id | customer_id |
+----------+-------------+
|    1     |      23     |
|    2     |      9      |
|    4     |      30     |
|    5     |      54     |
|    6     |      96     |
|    7     |      54     |
|    8     |      54     |
+----------+-------------+


In [3]:
display_table('Transactions')

+----------------+----------+--------+
| transaction_id | visit_id | amount |
+----------------+----------+--------+
|       2        |    5     |  310   |
|       3        |    5     |  300   |
|       9        |    5     |  200   |
|       12       |    1     |  910   |
|       13       |    2     |  970   |
+----------------+----------+--------+


### Write a SQL query to find the IDs of the users who visited without making any transactions and the number of times they made these types of visits.

```
+-------------+----------------+
| customer_id | count_no_trans |
+-------------+----------------+
| 54          | 2              |
| 30          | 1              |
| 96          | 1              |
+-------------+----------------+
```

In [24]:
%%sql 

-- customers who visited but did not make a purchase
SELECT customer_id, visit_id
FROM Visits 
WHERE visit_id NOT IN 
    (SELECT visit_id FROM Transactions)


customer_id,visit_id
30,4
96,6
54,7
54,8


In [27]:
%%sql 

SELECT customer_id, COUNT(customer_id) AS count_no_trans
FROM Visits
WHERE visit_id NOT IN 
    (SELECT visit_id FROM Transactions)
GROUP BY customer_id

customer_id,count_no_trans
30,1
96,1
54,2


# Using Pandas

In [28]:
visits_query = %sql SELECT * FROM Visits # type: ignore
transactions_query = %sql SELECT * FROM Transactions # type: ignore

visits_df = visits_query.DataFrame()
transactions_df = transactions_query.DataFrame()

display(visits_df, transactions_df)

,visit_id,customer_id
0,1,23
1,2,9
2,4,30
3,5,54
4,6,96
5,7,54
6,8,54


,transaction_id,visit_id,amount
0,2,5,310
1,3,5,300
2,9,5,200
3,12,1,910
4,13,2,970


## filter()

In [30]:
# visit_ids for customers who made a transaction
subquery_df = transactions_df['visit_id']
subquery_df

0    5
1    5
2    5
3    1
4    2
Name: visit_id, dtype: int64

In [31]:
# filter the visits_df to find customers who made a transaction
visits_df[visits_df['visit_id'].isin(subquery_df)]

,visit_id,customer_id
0,1,23
1,2,9
3,5,54


In [32]:
# customers who did not make a transaction 
visits_df[~visits_df['visit_id'].isin(subquery_df)]

,visit_id,customer_id
2,4,30
4,6,96
5,7,54
6,8,54


In [33]:
# group the non buyers by their_id
visits_df[~visits_df['visit_id'].isin(subquery_df)].groupby('customer_id')

In [34]:
visits_df[~visits_df['visit_id'].isin(subquery_df)] \
    .groupby('customer_id').size()

customer_id
30    1
54    2
96    1
dtype: int64

In [35]:
# rename the column
visits_df[~visits_df['visit_id'].isin(subquery_df)]. \
    groupby('customer_id').size(). \
        reset_index(name='count_no_trans')

,customer_id,count_no_trans
0,30,1
1,54,2
2,96,1


## merge()

In [38]:
# perform a left join on the 'visit_id' between 'visits_df' and 'transactions_df'
merged_df = visits_df.merge(transactions_df, on='visit_id', how='left')
merged_df

,visit_id,customer_id,transaction_id,amount
0,1,23,12.0,910.0
1,2,9,13.0,970.0
2,4,30,NaN,NaN
3,5,54,2.0,310.0
4,5,54,3.0,300.0
5,5,54,9.0,200.0
6,6,96,NaN,NaN
7,7,54,NaN,NaN
8,8,54,NaN,NaN


In [39]:
# filter out columns where 'transaction_id' is null 
merged_df[merged_df['transaction_id'].isna()]

,visit_id,customer_id,transaction_id,amount
2,4,30,NaN,NaN
6,6,96,NaN,NaN
7,7,54,NaN,NaN
8,8,54,NaN,NaN


In [41]:
# groupby 'customer_id' and 
merged_df[merged_df['transaction_id'].isna()]. \
    groupby('customer_id'). \
    size(). \
    reset_index(name='count_no_trans')

,customer_id,count_no_trans
0,30,1
1,54,2
2,96,1
